In [94]:
using Random
using StatsBase
using Formatting: printfmt

In [95]:
function parse_file(file)
    instancia = open("instancias/"*file)
    arr  = []
    for line in eachline(instancia)
        push!(arr, line)
    end
    close(instancia)
    
    vertices = []
    premios = []
    flag_vertices = true
    for line in arr[8:length(arr)]
        if startswith(line, "DEMAND_SECTION")
            flag_vertices = false
        elseif startswith(line, "DEPOT_SECTION")
            break
        elseif flag_vertices
            x, y = split(line)[2:3]
            x = parse(Int32, x)
            y = parse(Int32, y)
            push!(vertices, (x,y))
        else
            premio = split(line)[2]
            premio = parse(Int32, premio)
            push!(premios, premio)

        end
    end
    return (vertices, premios)
end


function get_distancias(vertices)
    cardinalidade = length(vertices)
    distancias = zeros(cardinalidade, cardinalidade)
    for i in 1:cardinalidade
        for j in 1:cardinalidade
            distancia = ((vertices[i][1] - vertices[j][1])^2 + (vertices[i][2] - vertices[j][2])^2)^(0.5)
            distancias[i, j] =  round(distancia)
        end
    end
    return distancias
end

get_distancias (generic function with 1 method)

In [96]:
function get_solucao_inicial(k)
    solucao_inicial = [1]
    vertices_nao_visitados = collect(2:k)
    return solucao_inicial, vertices_nao_visitados
end

function get_tabela_tabu_inicial(k) 
    return zeros(Int32, (2, k, k))
end

get_tabela_tabu_inicial (generic function with 1 method)

In [198]:
function pop_at(array, index)
    valor = array[index]
    deleteat!(array, index)
    return array, valor
end

function get_um_vizinho_a_direita(solucao, vertices_nao_visitados)
    direcao = 2
    
    # copias de seguranca
    vizinho = copy(solucao)
    copia_vertices_nao_visitados = copy(vertices_nao_visitados)
    
    # escolhe um vertice para colocar no caminho
    indice = rand(1:length(copia_vertices_nao_visitados))
    vertice_adicionado = copia_vertices_nao_visitados[indice]
    
    # deleta o vertice do array de vertices nao visitados
    deleteat!(copia_vertices_nao_visitados, indice)
    
    # escolhe um ponto de insercao do vertice na solucao
    indice_de_inclusao = rand(2:length(solucao)+1)
    
    # insere o vertice na solucao
    insert!(vizinho, indice_de_inclusao, vertice_adicionado)
    
    # calcula o custo associado a esse vizinho
    custo = get_custo(vizinho)
    
    # retorna vizinho, vertices nao visitados, tripla da tabela tabu
    return vizinho, custo, copia_vertices_nao_visitados, (direcao, vertice_adicionado, indice_de_inclusao)
end
    

# solucao deve ser uma lista de tamanho maior que 1
function get_um_vizinho_a_esquerda(solucao, vertices_nao_visitados) # solucao deve ser uma lista de tamanho maior que 1
    direcao = 1
    # copias de seguranca
    vizinho = copy(solucao)
    copia_vertices_nao_visitados = copy(vertices_nao_visitados)
    
    if length(solucao) == 1
        return vizinho, 0, copia_vertices_nao_visitados, (direcao, 1, 1)
        
    else
        # escolhe o indice do vertice no caminho para ser removido
        indice_de_exclusao = rand(2:length(solucao))

        # remove o vertice escolhido
        vizinho, vertice_deletado = pop_at(vizinho, indice_de_exclusao)

        # insere o vertice removido do caminho na array de vertices nao visitados
        push!(copia_vertices_nao_visitados, vertice_deletado);

        # calcula o custo associado a esse vizinho
        custo = get_custo(vizinho)
        
        # retorna vizinho, vertices nao visitados, tripla da tabela tabu
        return vizinho, custo, copia_vertices_nao_visitados, (direcao, vertice_deletado, 1)
    end
end

function get_vizinhos(solucao, vertices_nao_visitados)
    vizinhos = []
    for i in 1:trunc(Int32, sqrt(length(solucao)))
        vizinho = get_um_vizinho_a_esquerda(solucao, vertices_nao_visitados)
        if vizinho[1] != [1]
            push!(vizinhos, vizinho)
        end
    end
    for i in 1:trunc(Int32, length(vertices_nao_visitados) * 0.7)
        vizinho = get_um_vizinho_a_direita(solucao, vertices_nao_visitados)
        push!(vizinhos, vizinho)
    end
    return vizinhos
end

function get_custo(caminho)
    custo = 0
    for i in 1:length(caminho)-1
        saida = caminho[i]
        entrada = caminho[i+1]
        custo += D[saida, entrada] - P[entrada]
    end
    saida = last(caminho)
    entrada = 1
    custo += D[saida, entrada] - P[entrada] 
end

function movimento_eh_tabu(tabela_tabu, movimento, iteracao)
    direcao, vertice, indice = movimento
    return tabela_tabu[direcao, vertice, indice] > iteracao
end

function insere_movimento_na_tabela_tabu(tabela_tabu, movimento, iteracao)
    direcao, vertice, indice = movimento
    if direcao == 1 && vertice == 1 && vertice == 1
        tamanho_tabu = 9999999
    else
        tamanho_tabu = trunc(Int32, sqrt(size(tabela_tabu)[2]))
    end
    tabela_tabu[direcao, vertice, indice] = iteracao + tamanho_tabu
    return tabela_tabu
end      

insere_movimento_na_tabela_tabu (generic function with 1 method)

In [199]:
# solucao = [1, 3, 7, 4]
# vertices_nao_visitados = [2, 5, 6, 8]

# get_um_vizinho_a_direita(solucao, vertices_nao_visitados)

# solucao = [1]
# vertices_nao_visitados = [2,3,4,5,6,7,8]

# get_um_vizinho_a_esquerda(solucao, vertices_nao_visitados)

In [225]:
arquivo = "A-n80-k10.vrp"
v, P = parse_file(arquivo)
D = get_distancias(v)
k = length(P);
D = D[1:k, 1:k]
P = P[1:k];

In [226]:
solucao, vertices_nao_visitados  = get_solucao_inicial(k);
tabela_tabu = get_tabela_tabu_inicial(k);

In [227]:
menor_custo_global = 9999999
melhor_vizinho_global = nothing
for iteracao in 1:900000
    menor_custo_local = 9999999
    melhor_vizinho_local = nothing
    vizinhos = get_vizinhos(solucao, vertices_nao_visitados)
    for i in 1:length(vizinhos)
        vizinho = vizinhos[i]
        #printfmt("$vizinho[2], $\n\n")
        if vizinho[2] < menor_custo_local && !(movimento_eh_tabu(tabela_tabu, vizinho[4], iteracao))
            menor_custo_local = vizinho[2]
            melhor_vizinho_local = vizinho
        end
        
        if  vizinho[2] < menor_custo_global
            menor_custo_local = vizinho[2]
            melhor_vizinho_local = vizinho
            menor_custo_global = vizinho[2]
            melhor_vizinho_global = vizinho
        end
    end
    solucao = melhor_vizinho_local[1]
    vertices_nao_visitados = melhor_vizinho_local[3]
    tabela_tabu = insere_movimento_na_tabela_tabu(tabela_tabu, melhor_vizinho_local[4], iteracao)  
end
    

In [228]:
melhor_vizinho_global

([1, 50, 74, 14, 43, 52, 78, 4, 40, 61  …  80, 29, 53, 12, 64, 72, 11, 8, 2, 41], -208.0, [5, 10, 23, 33, 34, 36, 46, 51, 55, 57  …  44, 35, 30, 20, 22, 25, 54, 76, 37, 63], (2, 69, 29))

In [109]:
solucao, vertices_nao_visitados  = get_solucao_inicial(k);
tabela_tabu = get_tabela_tabu_inicial(k);
menor_custo_global = 9999999
melhor_vizinho_global = nothing

menor_custo_local = 9999999
melhor_vizinho_local = nothing
vizinhos = get_vizinhos(solucao, vertices_nao_visitados);

In [111]:
for i in 1:length(vizinhos)
    vizinho = vizinhos[i]
    if vizinho[2] < menor_custo_local && !(movimento_eh_tabu(tabela_tabu, vizinho[4], 0))
        menor_custo_local = vizinho[2]
        melhor_vizinho_local = vizinho
    end
        
    if  vizinho[2] < menor_custo_global
        menor_custo_local = vizinho[2]
        melhor_vizinho_local = vizinho
        menor_custo_global = vizinho[2]
        melhor_vizinho_global = vizinho
    end
end


In [112]:
melhor_vizinho_local

([1], 0, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  23, 24, 25, 26, 27, 28, 29, 30, 31, 32], (1, 1, 1))

In [114]:
solucao = melhor_vizinho_local[1]
vertices_nao_visitados = melhor_vizinho_local[3]
tabela_tabu = insere_movimento_na_tabela_tabu(tabela_tabu, melhor_vizinho_local[4], 0) 

2×32×32 Array{Int32,3}:
[:, :, 1] =
 5  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 2] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 3] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

...

[:, :, 30] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 31] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 32] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0 

In [158]:
arquivo = "A-n32-k5.vrp"
v, P = parse_file(arquivo)
D = get_distancias(v)
k = length(P);
k = 6 
D = D[1:k, 1:k]
P = P[1:k];

In [168]:
function get_custo(caminho, distancias, premios)
    custo = 0
    for i in 1:length(caminho)-1
        saida = caminho[i]
        entrada = caminho[i+1]
        custo += distancias[saida, entrada] - premios[entrada]
    end
    saida = last(caminho)
    entrada = 1
    custo += distancias[saida, entrada] - premios[entrada]
    return custo
end

get_custo (generic function with 2 methods)

In [169]:
caminho = [1, 4, 6, 3, 5, 2]

6-element Array{Int64,1}:
 1
 4
 6
 3
 5
 2

In [171]:
A = zeros(k, k)
V = zeros(k)
for i in 1:length(caminho)-1
    saida = caminho[i]
    entrada = caminho[i+1]
    A[saida, entrada] = 1
    V[entrada] = 1
    end
saida = last(caminho)
entrada = 1
A[saida, entrada] = 1
V[entrada] = 1

1

In [176]:
sum(A .* D) 

409.0

In [170]:
get_custo(caminho, D, P)

337.0

In [167]:
D[1,4] - P[4] + D[4,6] - P[6] + D[6,3] - P[3] + D[3,5] - P[5] + D[5,2] - P[2] + D[2,1] - P[1]

337.0

In [190]:
D

32×32 Array{Float64,2}:
   0.0   35.0   78.0  76.0  98.0   55.0  …  26.0  85.0   62.0  16.0   73.0
  35.0    0.0   60.0  59.0  91.0   81.0     46.0  79.0   80.0  19.0   39.0
  78.0   60.0    0.0   3.0  37.0   87.0     64.0  29.0   72.0  65.0   48.0
  76.0   59.0    3.0   0.0  36.0   83.0     62.0  27.0   68.0  63.0   49.0
  98.0   91.0   37.0  36.0   0.0   84.0     76.0  13.0   63.0  89.0   85.0
  55.0   81.0   87.0  83.0  84.0    0.0  …  34.0  74.0   21.0  63.0  109.0
  52.0   40.0   26.0  24.0  51.0   66.0     39.0  38.0   55.0  40.0   47.0
  37.0   13.0   48.0  47.0  78.0   74.0     40.0  66.0   71.0  21.0   37.0
  86.0   84.0   41.0  38.0  17.0   67.0     62.0  13.0   46.0  80.0   86.0
  88.0   94.0   59.0  56.0  34.0   57.0     63.0  32.0   36.0  86.0  102.0
  79.0  100.0   90.0  87.0  76.0   27.0  …  56.0  70.0   21.0  85.0  122.0
 101.0   97.0   45.0  44.0   9.0   83.0     79.0  19.0   62.0  94.0   93.0
  29.0    8.0   67.0  66.0  96.0   78.0     44.0  84.0   80.0  15.0   47.0
 